# Interpreting Machine Learning models using LIME and SHAP
This notebook briefly shows how to use LIME and SHAP to explain any kind of AI model.

We will first cover LIME and then we will move on to SHAP.

## LIME
Import LIME and Vader

In [ ]:
# Import vader model and LIME for text
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from lime.lime_text import LimeTextExplainer

# Import numpy for formatting
import numpy as np

The object containing the trained model must be compatible with LIME. In particular, the object needs to have a `predict_proba` method.

Hence, we'll add a new method to the Vader class.

In [ ]:
# Declare a function that can score multiple texts
def predict_proba(self, texts):

  # Initialize empty list
  ret = []

  # Iterate over texts
  for text in texts:

    # Get negative score
    neg = self.polarity_scores(text).get('neg')

    # Return two outputs: (neg) and (1 - neg)
    ret.append([neg, 1 - neg])

  # Return predictions
  return np.array(ret)

# Add the function as a method
SentimentIntensityAnalyzer.predict_proba = predict_proba

# Instantiate model
vader = SentimentIntensityAnalyzer()

Try on text

In [ ]:
# Declare text
my_angry_text = 'I HATE Mondays! I missed the bus and forgot my lunch...'

# Instantiate explainer
explainer = LimeTextExplainer(class_names=['negative', 'not-negative'])

# Create explanation for text
explanation = explainer.explain_instance(
    my_angry_text,
    vader.predict_proba,
    num_features=4
)

# Show explanation
explanation.show_in_notebook()

# SHAP

Import SHAP

In [ ]:
%%capture

# Install shap
!pip install shap

# Import dependencies
import shap
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression

In [ ]:
# Load features as pandas dataframe
X = pd.DataFrame(
    data=load_diabetes()['data'],
    columns=load_diabetes()['feature_names']
)

# Load target
y = load_diabetes()['target']

# Instantiate model
model = LinearRegression()

# Fit model to data
model.fit(X=X, y=y)

Partial dependence plot

In [ ]:
# Draw sample of 200 observations
sample = shap.utils.sample(X=X, nsamples=200)

# Partial dependence plot for body-mass index variable
shap.partial_dependence_plot(
    ind='bmi',
    model=model.predict,
    data=sample,
    ice=False,
    model_expected_value=True,
    feature_expected_value=True
)

Explain 42nd observation.

In [ ]:
print(sample.iloc[[42], :])

In [ ]:
explainer = shap.Explainer(
    model=model.predict
)

In [ ]:
# Instantiate explainer
explainer = shap.Explainer(
    model=model.predict,
    masker=sample
)

# Calculate SHAP values
shap_values = explainer(sample)

# Pick 42nd observation from sample to explain
idx = 42

# Partial dependence plot for body-mass index variable
shap.partial_dependence_plot(
    ind='bmi',
    model=model.predict,
    data=sample,
    ice=False,
    model_expected_value=True,
    feature_expected_value=True,
    hist=True,
    shap_values=shap_values[idx:idx+1]
)

In [ ]:
shap.plots.waterfall(shap_values[idx])

In [ ]:
model.predict(sample.iloc[[idx], :])